In [1]:
import os
import openai
import json
import requests
import pytz
from sqldbconn import get_connx
from datetime import datetime
from collections import OrderedDict 
# import Markdown to display the results
from IPython.display import Markdown, display

from dotenv import load_dotenv

env_path = r"C:\Users\hyssh\workspace\openai-demo\quickstart-learnfast\function-calling\.env"
load_dotenv(dotenv_path=env_path, override=True)

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
ENGINE = os.getenv("ENGINE")
bing_search_url = os.getenv("BING_SEARCH_API_BASE")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_KEY")
search_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
search_api_version = os.getenv("AZURE_SEARCH_API_VERSION")

In [2]:
def search_bing(query):
    headers = {"Ocp-Apim-Subscription-Key": os.getenv("BING_SEARCH_API_KEY")}
    params = {"q": query, "textDecorations": False }
    response = requests.get("https://api.bing.microsoft.com/v7.0/search", headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results['webPages']['value']:
        output.append({
            'title': result['name'],
            'link': result['url'],
            'snippet': result['snippet']
        })

    return json.dumps(output)

def pci_dss_v4(query)->str:
    headers = {'Content-Type': 'application/json','api-key': search_key}
    params = {'api-version': search_api_version}
    k = 5
    reranker_threshold =1

    agg_search_results = dict()

    search_payload = {
        "search": query,
        "queryType": "semantic",
        "semanticConfiguration": "my-semantic-config",
        "count": "true",
        "speller": "lexicon",
        "queryLanguage": "en-us",
        "captions": "extractive",
        "answers": "extractive",
        "top": k
    }

    search_payload["select"]= "id, title, content, name, location"
    

    resp = requests.post(search_endpoint + "/indexes/" + search_index_name + "/docs/search",
                        data=json.dumps(search_payload), headers=headers, params=params)

    search_results = resp.json()
    agg_search_results[index] = search_results

    content = dict()
    ordered_content = OrderedDict()
    
    for index,search_results in agg_search_results.items():
        for result in search_results['value']:
            if result['@search.rerankerScore'] > reranker_threshold: # Show results that are at least N% of the max possible score=4
                content[result['id']]={
                                        "title": result['title'], 
                                        "name": result['name'], 
                                        "location": result['location'],
                                        "caption": result['@search.captions'][0]['text'],
                                        "index": index
                                    }
                content[result['id']]["content"]= result['content']
                content[result['id']]["score"]= result['@search.score'] # Uses the Hybrid RRF score
                
    # After results have been filtered, sort and add the top k to the ordered_content
        
    count = 0  # To keep track of the number of results added
    for id in sorted(content, key=lambda x: content[x]["score"], reverse=True):
        ordered_content[id] = content[id]
        count += 1
        if count >= k:  # Stop after adding k results
            break

    return json.dumps(ordered_content)

def retrieve_sales_report(query)->str:
    # Your implementation here
    import json
    import pandas as pd
    from sqldbconn import get_connx


    sql_query = """
    SELECT d.[Calendar Year], d.[Calendar Month Number], sum(o.[Total Excluding Tax]) as [Total Order Amount]
    FROM [Fact].[Order] as o JOIN [Dimension].[Date] as d ON o.[Order Date Key] = d.[Date]
    GROUP BY d.[Calendar Year], d.[Calendar Month Number]
    ORDER BY d.[Calendar Year], d.[Calendar Month Number]
    """
    df = pd.read_sql_query(sql_query, get_connx())
    return json.dumps(df.to_dict(orient='records'))

def sales_forecast(dataset)->str:
    sample_sales_forecast = """
[
  {"Year": 2016, "Month": 6, "Total Order Amount": 5345009.39},
  {"Year": 2016, "Month": 7, "Total Order Amount": 5209132.57},
  {"Year": 2016, "Month": 8, "Total Order Amount": 5217514.01}
]
    """
    return json.dumps(sample_sales_forecast)


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [3]:
# Check functions
# get_current_time("Seattle")
# search_bing("where will the 2032 olymbics be held?")

In [4]:
functions = [  
    {
        "name": "search_bing",
        "description": "Searches bing to get up to date information from the web",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query",
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "retrieve_sales_report",
        "description": "Retrieves sales report from data warehouse that has sales data",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Sales report title and description",
                }
            },
            "required": ["query"],
        },
        "output":{
            "type": "object",
            "name": "sales report",
            "properties": {
                "dataset": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "Year": {
                            "type": "number",
                            "description": "The year of the sales data"
                        },
                        "Month": {
                            "type": "number",
                            "description": "The month of the sales data"
                        },
                        "Profits($M)": {
                            "type": "number",
                            "description": "The profits in millions for the given month and year"
                        }
                    },
                    "required": ["Year", "Month", "Profits($M)"],
                    "description": "An object representing sales data for a specific month and year"
                },
                "description": "The dataset for the sales"
            }
            },
        }
    },
    {
        "name": "sales_forecast",
        "description": "Makes a forecast on sales based on given dataset, if there is a sales report, use that as input, if there is no sales report, abort the forecast and return error",
        "parameters": {
        "type": "object",
        "properties": {
            "dataset": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "Year": {
                            "type": "number",
                            "description": "The year of the sales data"
                        },
                        "Month": {
                            "type": "number",
                            "description": "The month of the sales data"
                        },
                        "Profits($M)": {
                            "type": "number",
                            "description": "The profits in millions for the given month and year"
                        }
                    },
                    "required": ["Year", "Month", "Profits($M)"],
                    "description": "An object representing sales data for a specific month and year"
                },
                "description": "The dataset for the sales"
            }
        },
        "required": ["dataset"]
        }
    },
    {
        "name": "get_current_time",
        "description": "Get the current time in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                }
            },
            "required": ["location"],
        },
    },
    {
        "name": "pci_dss_v4",
        "description": "Searches the PCI DSS v4.0 document to get up to date information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "PCI DSS related search query",
                }
            },
            "required": ["query"],
        },
    },
]

In [5]:
available_functions = {
    "search_bing": search_bing,
    "retrieve_sales_report": retrieve_sales_report,
    "sales_forecast": sales_forecast,
    "get_current_time":get_current_time,
    "pci_dss_v4": pci_dss_v4
}

In [6]:
def run_multiturn_conversation(messages, functions, available_functions, deployment_name, verbose: bool=False):
    """
    Runs a multi-turn conversation with GPT4, where GPT4 may call functions
    :param messages: a list of messages in the conversation, where each message is a dict with keys "role" and "content"
    :param functions: a list of functions that GPT4 can call
    :param available_functions: a dict of function names to functions
    :param deployment_name: the name of the deployment to use


    """
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_name,
        messages=messages,
        functions=functions,
        function_call="auto", 
        temperature=0
    )

    if verbose:
        print(response)

    # Step 2: check if GPT wanted to call a function
    while response["choices"][0]["finish_reason"] == 'function_call':
        if verbose:
            print("== Response Type: function_call {} ==".format(response["choices"][0]["finish_reason"]))
        
        response_message = response["choices"][0]["message"]
        
        if verbose:
            print("Recommended Function call:")
            print(response_message.get("function_call"))
            print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(**function_args)
        
        if verbose:
            print(function_name, "Output of function call:", function_response)
            print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        if verbose:
            print("== Messages in next request:")
            for message in messages:
                print(message)
            print("=============================")

        response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_name,
            function_call="auto",
            functions=functions,
            temperature=0
        )  # get a new response from GPT where it can see the function response

        if verbose:
            print(response)

    return response, messages


In [7]:
user_message = """
Show me the sales report for the latest quarter. And also show me a forecast based on the data in the sales report
"""

# user_message = """
# Get me the sales report for the leatest quarter.
# """

# user_message = """
# What time is in now in Seoul, South Korea
# """

# user_message = """
# What is the latest date in sales report and how many days are aport today's date in Seattle timezone?
# """

# user_message = """
# Latest price of bitcoin?
# """

# user_message = """
# what should i check to make sure i'm complying with PCI DSS for database protection?
# """

In [8]:
system_message = """You are an assistant designed to help business analyist and data analyist answer questions.
You have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.
You have access to data warehouse that has sales data. You should call retrieve_sales_report whenever a question requires historical sales data.
You have access to a forecasting model that can make predictions on sales. You should call sales_forecast whenever a question requires sales forecast. Make sure to use output from retrieve_sales_report as input for sales_forecast.
You have access to PCI DSS v4 document. You should call pci_dss_v4 whenever a question requires information from PCI DSS v4 document.

In your answer, repeast the question and then call the appropriate function to get the answer.

## Safety

If you don't have funtion or skill to answer a question, say you don't know. 

## Response

If you used a Function Calling state that which function you used
Reponse format
[Funtion `function_name` called]

[Your ansers]

### Example 1
[Function `search_bing` called] 

The latest price of Bitcoin is approximately $38,808.75 USD

### Example 2
[Function `retrieve_sales_report` called]

Latest date in the sales report is from May 2016. 

"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": user_message}]


result, conversations = run_multiturn_conversation(messages, functions, available_functions, ENGINE, verbose=False)

# print("Final response:")
print(result['choices'][0]['message']['content'])

C:\Users\hyssh\AppData\Local\Temp\ipykernel_13240\2004228846.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, get_connx())


[Function `retrieve_sales_report` called]

The sales report for the latest quarter is as follows:
- March 2016: $4,807,110.70
- April 2016: $4,739,058.60
- May 2016: $5,138,002.65

[Function `sales_forecast` called]

Based on the data in the sales report, the sales forecast for the next quarter is:
- June 2016: $5,345,009.39
- July 2016: $5,209,132.57
- August 2016: $5,217,514.01


In [9]:
str_conversation = f"""```json
{json.dumps(conversations, indent=1)} 
```"""
#display result well formatted in markdown
display(Markdown(str_conversation))

```json
[
 {
  "role": "system",
  "content": "You are an assistant designed to help business analyist and data analyist answer questions.\nYou have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.\nYou have access to data warehouse that has sales data. You should call retrieve_sales_report whenever a question requires historical sales data.\nYou have access to a forecasting model that can make predictions on sales. You should call sales_forecast whenever a question requires sales forecast. Make sure to use output from retrieve_sales_report as input for sales_forecast.\nYou have access to PCI DSS v4 document. You should call pci_dss_v4 whenever a question requires information from PCI DSS v4 document.\n\nIn your answer, repeast the question and then call the appropriate function to get the answer.\n\n## Safety\n\nIf you don't have funtion or skill to answer a question, say you don't know. \n\n## Response\n\nIf you used a Function Calling state that which function you used\nReponse format\n[Funtion `function_name` called]\n\n[Your ansers]\n\n### Example 1\n[Function `search_bing` called] \n\nThe latest price of Bitcoin is approximately $38,808.75 USD\n\n### Example 2\n[Function `retrieve_sales_report` called]\n\nLatest date in the sales report is from May 2016. \n\n"
 },
 {
  "role": "user",
  "content": "\nShow me the sales report for the latest quarter. And also show me a forecast based on the data in the sales report\n"
 },
 {
  "role": "assistant",
  "function_call": {
   "name": "retrieve_sales_report",
   "arguments": "{\n  \"query\": \"latest quarter sales report\"\n}"
  },
  "content": null
 },
 {
  "role": "function",
  "name": "retrieve_sales_report",
  "content": "[{\"Calendar Year\": 2013, \"Calendar Month Number\": 1, \"Total Order Amount\": 3824842.85}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 2, \"Total Order Amount\": 2821282.2}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 3, \"Total Order Amount\": 3966078.1}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 4, \"Total Order Amount\": 4155710.05}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 5, \"Total Order Amount\": 4562830.35}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 6, \"Total Order Amount\": 4150098.6}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 7, \"Total Order Amount\": 4502741.85}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 8, \"Total Order Amount\": 3601220.6}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 9, \"Total Order Amount\": 3916003.25}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 10, \"Total Order Amount\": 3879872.45}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 11, \"Total Order Amount\": 3819809.1}, {\"Calendar Year\": 2013, \"Calendar Month Number\": 12, \"Total Order Amount\": 3728103.4}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 1, \"Total Order Amount\": 4202578.8}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 2, \"Total Order Amount\": 3572744.4}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 3, \"Total Order Amount\": 3955257.55}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 4, \"Total Order Amount\": 4212856.25}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 5, \"Total Order Amount\": 4753224.1}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 6, \"Total Order Amount\": 4427573.8}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 7, \"Total Order Amount\": 4919791.85}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 8, \"Total Order Amount\": 4197257.4}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 9, \"Total Order Amount\": 3973877.85}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 10, \"Total Order Amount\": 4606478.45}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 11, \"Total Order Amount\": 4157270.55}, {\"Calendar Year\": 2014, \"Calendar Month Number\": 12, \"Total Order Amount\": 4513092.4}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 1, \"Total Order Amount\": 4556065.25}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 2, \"Total Order Amount\": 4307819.25}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 3, \"Total Order Amount\": 4644642.35}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 4, \"Total Order Amount\": 5222594.85}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 5, \"Total Order Amount\": 4636628.45}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 6, \"Total Order Amount\": 4696866.65}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 7, \"Total Order Amount\": 5339212.0}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 8, \"Total Order Amount\": 4070526.6}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 9, \"Total Order Amount\": 4841896.7}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 10, \"Total Order Amount\": 4653840.9}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 11, \"Total Order Amount\": 4240612.3}, {\"Calendar Year\": 2015, \"Calendar Month Number\": 12, \"Total Order Amount\": 4607182.15}, {\"Calendar Year\": 2016, \"Calendar Month Number\": 1, \"Total Order Amount\": 4612140.45}, {\"Calendar Year\": 2016, \"Calendar Month Number\": 2, \"Total Order Amount\": 4099480.35}, {\"Calendar Year\": 2016, \"Calendar Month Number\": 3, \"Total Order Amount\": 4807110.7}, {\"Calendar Year\": 2016, \"Calendar Month Number\": 4, \"Total Order Amount\": 4739058.6}, {\"Calendar Year\": 2016, \"Calendar Month Number\": 5, \"Total Order Amount\": 5138002.65}]"
 },
 {
  "role": "assistant",
  "function_call": {
   "name": "sales_forecast",
   "arguments": "{\n  \"dataset\": [\n    {\"Year\": 2016, \"Month\": 3, \"Profits($M)\": 4807110.7},\n    {\"Year\": 2016, \"Month\": 4, \"Profits($M)\": 4739058.6},\n    {\"Year\": 2016, \"Month\": 5, \"Profits($M)\": 5138002.65}\n  ]\n}"
  },
  "content": null
 },
 {
  "role": "function",
  "name": "sales_forecast",
  "content": "\"\\n[\\n  {\\\"Year\\\": 2016, \\\"Month\\\": 6, \\\"Total Order Amount\\\": 5345009.39},\\n  {\\\"Year\\\": 2016, \\\"Month\\\": 7, \\\"Total Order Amount\\\": 5209132.57},\\n  {\\\"Year\\\": 2016, \\\"Month\\\": 8, \\\"Total Order Amount\\\": 5217514.01}\\n]\\n    \""
 }
] 
```

## Next step

Bring the code to an app using Gradio

![](../../images/FuntionCalling_Chat.png)

In [10]:
# End of notebook